In [1]:
from random import randint
from time import sleep
import os
import numpy as np
from openai import OpenAI
import pandas as pd

C:\Users\user\.conda\envs\chatgpt\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\.conda\envs\chatgpt\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\user\.conda\envs\chatgpt\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Load Twitter data containing tweets

In [2]:
tweets_file_name = "./example_synthetic_tweets.csv"
tweets_df = pd.read_csv(tweets_file_name)
tweets_df.head()

,author_id,created_at,id,post,retweet_count,reply_count,like_count,quote_count,impression_count,engagement_count,clean_post,clean_post_for_sentiment,gpt_sentiment,gpt_emotion
0,2.270260e+11,2023-02-09T18:53:48.000Z,162375558542660419,“Exploring the world of AI with #ChatGPT. It’s...,0,0,0,0,13,13,exploring world ai with chatgpt having convers...,Exploring the world of AI with ChatGPT. It’s l...,NaN,NaN
1,2.125991e+09,2023-02-09T18:49:46.000Z,162375558542665819,“Just had a fascinating chat with #ChatGPT. It...,0,0,0,0,32,32,fascinating chat chatgpt amazing how far ai te...,Just had a fascinating chat with ChatGPT. It’s...,NaN,NaN
2,5.168851e+09,2023-02-09T18:41:16.000Z,162375398857466630,“Need a buddy to chat with? Try #ChatGPT. You’...,0,0,0,0,35,35,buddy to chat try chatgpt amazed depth convers...,Need a buddy to chat with? Try ChatGPT. You'll...,NaN,NaN
3,1.532780e+11,2023-02-09T18:29:15.000Z,16237507541208583681,“AI is transforming the way we communicate. #C...,0,1,0,0,30,31,ai transforming way communicate chatgpt leadin...,AI is transforming the way we communicate. Cha...,NaN,NaN
4,3.001470e+13,2023-02-09T18:23:02.000Z,162374933584554624,“Who knew chatting with an AI could be so enli...,0,0,0,0,9,9,knew chatting ai enlightening chatgpt continue...,Who knew chatting with an AI could be so enlig...,NaN,NaN


## Set up the OpenAI API client

In [3]:
# Set up the OpenAI API client
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # OPENAI - API KEY
client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)

## Get sentiment of tweets from ChatGPT

In [4]:
sentiments_dict = {1: "positive", 2: "neutral", 3: "negative"}
def sentiment_prompt(text):
    # Add random delay to the function to avoid overload
    delay = randint(1, 10)
    sleep(delay)
    model_engine = "gpt-3.5-turbo"
    prompt = [{"role": "user",
               "content": "Is the sentiment of this tweet positive, neutral, or negative? Answer only with a number: 1 if {}, 2 if {}, and 3 if {}. Here is the text:".format(
                   sentiments_dict[1],
                   sentiments_dict[2],
                   sentiments_dict[3])
              },
              {"role": "user", "content": text}]
    
    # Generate a response
    chat_completion = client.chat.completions.create(
        messages=prompt,
        model="gpt-3.5-turbo",
        temperature=0.5,
        max_tokens=1024
    )
    
    response = chat_completion.choices[0].message.content
    return response

In [5]:
for i, row in tweets_df.iterrows():
    tweet_id = row["id"]
    if pd.isnull(row.gpt_sentiment):
        try:
            tweets_df.loc[i,'gpt_sentiment'] = sentiment_prompt(row.clean_post_for_sentiment)
            print(row.clean_post_for_sentiment)
            print("Sentiment of tweet {} is **{} ({})**".format(tweet_id, tweets_df.at[i,'gpt_sentiment'], sentiments_dict[int(tweets_df.at[i,'gpt_sentiment'])]))
            print("===============================")
        except Exception as e:
            print("Somthing went wrong with tweet ID", tweet_id, e)

Exploring the world of AI with ChatGPT. It’s like having a conversation with the future!
Sentiment of tweet 162375558542660419 is **1 (positive)**
Just had a fascinating chat with ChatGPT. It’s amazing how far AI technology has come.
Sentiment of tweet 162375558542665819 is **1 (positive)**
Need a buddy to chat with? Try ChatGPT. You'll be amazed at the depth of the conversation.
Sentiment of tweet 162375398857466630 is **1 (positive)**
AI is transforming the way we communicate. ChatGPT is leading the charge with engaging and intelligent conversations.
Sentiment of tweet 16237507541208583681 is **1 (positive)**


## Get emotion of tweets from ChatGPT

In [6]:
emotions_dict = {1: "anger", 2: "joy", 3: "sarcasm", 4: "fear", 5: "surprise", 6: "sadness", 7: "disgust", 8: "neutral"}

def emotion_prompt(text):
    # Add random delay to the function to avoid overload
    delay = randint(1,5)
    sleep(delay)

    model_engine = "gpt-3.5-turbo"
    prompt = [{"role": "user",
               "content": f"Which of these emotions - anger, joy, sarcasm, fear, surprise, sadness, disgust, neutral - best represents the mental state of the person writing the following tweet? Answer only with a number: 1 if {emotions_dict[1]}, 2 if {emotions_dict[2]}, 3 if {emotions_dict[3]}, 4 if {emotions_dict[4]}, 5 if {emotions_dict[5]}, 6 if {emotions_dict[6]}, 7 if {emotions_dict[7]}, 8 if {emotions_dict[8]}. Here is the text:"},
              {"role": "user", "content": text}]

    # Generate a response
    chat_completion = client.chat.completions.create(
        messages=prompt,
        model="gpt-3.5-turbo",
        temperature=0.5,
        max_tokens=1024
    )
    
    response = chat_completion.choices[0].message.content
    return response

In [7]:
for i, row in tweets_df.iterrows():
    tweet_id = row["id"]
    if pd.isnull(row.gpt_emotion):
        try:
            tweets_df.loc[i,'gpt_emotion'] = emotion_prompt(row.clean_post_for_sentiment)
            print(row.clean_post_for_sentiment)
            print("Emotion of tweet {} is **{} ({})**".format(tweet_id, tweets_df.loc[i,'gpt_emotion'], emotions_dict[int(tweets_df.loc[i,'gpt_emotion'])]))
            print("===============================")
        except Exception as e:
            print("Somthing went wrong with tweet ID", tweet_id, e)

Exploring the world of AI with ChatGPT. It’s like having a conversation with the future!
Emotion of tweet 162375558542660419 is **2 (joy)**
Just had a fascinating chat with ChatGPT. It’s amazing how far AI technology has come.
Emotion of tweet 162375558542665819 is **2 (joy)**
Need a buddy to chat with? Try ChatGPT. You'll be amazed at the depth of the conversation.
Emotion of tweet 162375398857466630 is **2 (joy)**
AI is transforming the way we communicate. ChatGPT is leading the charge with engaging and intelligent conversations.
Emotion of tweet 16237507541208583681 is **2 (joy)**
Somthing went wrong with tweet ID 162374933584554624 Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-SCwYFtUUEyk8pfsUMNAm0Az0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at htt

## Get topic groups for tweets from ChatGPT

In [8]:
def classify_prompt(text):
    # Add random delay to the function to avoid overload
    delay = randint(1, 5)
    sleep(delay)

    model_engine = "gpt-3.5-turbo"

    prompt = [{"role": "user", "content": "Which one of this topics best describes the following tweet? Answer only with the topic number. \
               Topic 1: ChatGPT's Description, Functionality, and Applications. \
               Topic 2: Criticism, Concerns, and Limitations of ChatGPT. \
               Topic 3: AI in Education, Employment, and Various Industries. \
               Topic 4: Comparison, Integration, and Advancements of AI Technologies. \
               Topic 5: Ethics, Bias, and Regulation of AI. \
               If none of the topics is suitable, write the word 'DIFFERENT'. \
               The tweet:"},
              {"role": "user", "content": text}]

    # Generate a response
    chat_completion = client.chat.completions.create(
        messages=prompt,
        model="gpt-3.5-turbo",
        temperature=0.6,
        max_tokens=1024
    )

    response = chat_completion.choices[0].message.content
    return response

In [11]:
tweets_df["gpt_classify"] = np.nan

print("Starting execution....")
for i, row in tweets_df.iterrows():
    tweet_id = row["id"]
    if pd.isnull(row.gpt_classify):
        try:
            response = classify_prompt(row.clean_post_for_sentiment)
            tweets_df.loc[i,'gpt_classify'] = response
            print(row.clean_post_for_sentiment)
            print("Topic of tweet {} is **{}**".format(tweet_id, tweets_df.at[i,'gpt_classify']))
            print("===============================")
        except Exception as e:
            continue
            # print("Somthing went wrong with tweet ID", tweet_id, e)

Starting execution....
Exploring the world of AI with ChatGPT. It’s like having a conversation with the future!
Topic of tweet 162375558542660419 is **Topic 1: ChatGPT's Description, Functionality, and Applications.**
Just had a fascinating chat with ChatGPT. It’s amazing how far AI technology has come.
Topic of tweet 162375558542665819 is **Topic 1: ChatGPT's Description, Functionality, and Applications.**
Need a buddy to chat with? Try ChatGPT. You'll be amazed at the depth of the conversation.
Topic of tweet 162375398857466630 is **Topic 1: ChatGPT's Description, Functionality, and Applications.**
Who knew chatting with an AI could be so enlightening? ChatGPT continues to impress with its knowledge and wit.
Topic of tweet 162374933584554624 is **Topic 1: ChatGPT's Description, Functionality, and Applications.**


In [12]:
tweets_df

,author_id,created_at,id,post,retweet_count,reply_count,like_count,quote_count,impression_count,engagement_count,clean_post,clean_post_for_sentiment,gpt_sentiment,gpt_emotion,gpt_classify
0,2.270260e+11,2023-02-09T18:53:48.000Z,162375558542660419,“Exploring the world of AI with #ChatGPT. It’s...,0,0,0,0,13,13,exploring world ai with chatgpt having convers...,Exploring the world of AI with ChatGPT. It’s l...,NaN,2,"Topic 1: ChatGPT's Description, Functionality,..."
1,2.125991e+09,2023-02-09T18:49:46.000Z,162375558542665819,“Just had a fascinating chat with #ChatGPT. It...,0,0,0,0,32,32,fascinating chat chatgpt amazing how far ai te...,Just had a fascinating chat with ChatGPT. It’s...,NaN,2,"Topic 1: ChatGPT's Description, Functionality,..."
2,5.168851e+09,2023-02-09T18:41:16.000Z,162375398857466630,“Need a buddy to chat with? Try #ChatGPT. You’...,0,0,0,0,35,35,buddy to chat try chatgpt amazed depth convers...,Need a buddy to chat with? Try ChatGPT. You'll...,NaN,2,"Topic 1: ChatGPT's Description, Functionality,..."
3,1.532780e+11,2023-02-09T18:29:15.000Z,16237507541208583681,“AI is transforming the way we communicate. #C...,0,1,0,0,30,31,ai transforming way communicate chatgpt leadin...,AI is transforming the way we communicate. Cha...,NaN,2,NaN
4,3.001470e+13,2023-02-09T18:23:02.000Z,162374933584554624,“Who knew chatting with an AI could be so enli...,0,0,0,0,9,9,knew chatting ai enlightening chatgpt continue...,Who knew chatting with an AI could be so enlig...,NaN,NaN,"Topic 1: ChatGPT's Description, Functionality,..."


In [ ]:
tweets_df.to_csv("./example_synthetic_tweets_with_sentiment_emotion.csv")
print("Finished!")